In [68]:
import IPython
import pathlib
from PIL import Image
import skunk
import re
import json
import matplotlib.pyplot as plt
import xmltodict
from annotation_qc import get_surface_image_and_insertion_json_paths

In [69]:
surface_image_path, insertion_json_path = get_surface_image_and_insertion_json_paths('670181_20230718', templeton=True)
surface_image = Image.open(surface_image_path)

holes_letters: dict = {}

with open(insertion_json_path) as f:
    insertion_json = json.load(f)['probe_insertions']

for probe in insertion_json:
    if 'probe' in probe:
        probe_dict = insertion_json[probe]
        holes_letters[probe_dict['hole']] = probe_dict['letter']
holes_letters

{'A3': 'A', 'B1': 'B', 'B2': 'C', None: 'D', 'F2': 'E', 'F1': 'F'}

In [70]:
#TODO: get right one depending on templeton or dynamic routing, right now just hard-coded for templeton
probe_hole_idx = {
    "A": [1, 2, 3],
    "B": [1, 2, 3],
    "C": [1, 2, 3, 4],
    "D": [1,],
    "E": [],
    "F": [1, 2],
}
"1-indexed hole indices available for each probe."
hole_labels: tuple[str, ...] = tuple(
    f"{probe}{index}"
    for probe, indices in probe_hole_idx.items()
    for index in indices
)

In [71]:
hole_labels

('A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3', 'C4', 'D1', 'F1', 'F2')

In [78]:
data = pathlib.Path(r"\\allen\programs\mindscope\workgroups\dynamicrouting\arjun\Templeton_probes.svg").read_text()

In [79]:
xml_dict = xmltodict.parse(data)

holes_texts = xml_dict['svg']['g']['text']
for hole_text in holes_texts:
    hole = hole_text['tspan']['#text']
    if hole in hole_labels:
        print(hole, hole_text['@x'], hole_text['@y'])

B1 459.06863 790.37988
A1 570.71246 574.79358
B3 526.31543 914.69263
C4 333.16177 849.28333
B2 200.82866 763.24213
C2 169.83687 890.87549
C3 493.2132 692.13873
C1 275.67654 943.37231
F2 366.8978 222.92195
D1 179.09305 975.4798
F1 459.67792 189.43776
A2 565.18213 427.8959
A3 548.48883 244.41785


In [74]:
for hole in hole_labels:
    if hole in holes_letters:
        data = data.replace(
            f">{hole}</tspan>", f"> {holes_letters[hole]}</tspan>"
        )
    else:
        data = data.replace(f">{hole}</tspan>", f">""</tspan>")


In [75]:
fig, ax = plt.subplots(ncols=2)
fig.set_size_inches(16, 8)
ax[0].imshow(surface_image)
skunk.connect(ax[1], 'sk')
svg = skunk.insert({'sk': data})
plt.close()

In [76]:
skunk.display(svg)